In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!python --version

Python 3.10.18


In [1]:
import sys
print(sys.prefix)
print(sys.base_prefix)

c:\Users\User\anaconda3\envs\myenv_tf
c:\Users\User\anaconda3\envs\myenv_tf


In [4]:
from transformers import pipeline

# Load summarizer pipeline
summarizer = pipeline("summarization", model="google/pegasus-xsum")

def summarize_news(text):
    summary = summarizer(text, max_length=60, min_length=10, do_sample=False)
    return summary[0]['summary_text']


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [5]:
news = "On August 13, 2025, the cryptocurrency exchange operator Bullish debuted on the New York Stock Exchange, opening at $90—an astonishing 143% above its IPO price of $37. The company raised $1.11 billion, significantly outperforming its initial target range of $28–31 per share. Led by former NYSE president Tom Farley and owning CoinDesk, Bullish's successful launch marks a renewed investor interest in crypto-sector IPOs."
summarize_news(news)

"It's been 10 years since the first Initial Public Offering (IPO) in the crypto sector."

In [7]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import os

class LocalNewsSummarizer:
    def __init__(self, model_name="google/pegasus-xsum", local_path="./models/pegasus-xsum"):
        self.model_name = model_name
        self.local_path = local_path
        self.summarizer = None
        
        # Create models directory if it doesn't exist
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        
        self._load_or_download_model()
    
    def _load_or_download_model(self):
        if os.path.exists(self.local_path):
            print(f"Loading model from {self.local_path}...")
            self.summarizer = pipeline("summarization", model=self.local_path)
            print("Local model loaded successfully!")
        else:
            print(f"Local model not found. Downloading {self.model_name}...")
            self._download_and_save_model()
            self.summarizer = pipeline("summarization", model=self.local_path)
            print("Model downloaded and loaded!")
    
    def _download_and_save_model(self):
        # Download tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        
        # Save to local directory
        tokenizer.save_pretrained(self.local_path)
        model.save_pretrained(self.local_path)
        print(f"Model saved to {self.local_path}")
    
    def summarize(self, text):
        if self.summarizer is None:
            raise ValueError("Model not loaded!")
        
        summary = self.summarizer(text, max_length=60, min_length=10, do_sample=False)
        return summary[0]['summary_text']

# Usage
summarizer = LocalNewsSummarizer()
news_text = """
India on Saturday became the first country to successfully land a spacecraft near the Moon's south pole. 
The Chandrayaan-3 mission marks a major milestone in India's space exploration program, 
opening up new opportunities for scientific research and potential resource utilization. 
Prime Minister Narendra Modi congratulated the scientists and hailed it as a historic moment.
"""

summary = summarizer.summarize(news_text)
print("Summary:", summary)

Loading model from ./models/pegasus-xsum...


Device set to use cpu


Local model loaded successfully!
Summary: India's space agency, the Indian Space Research Organisation (Isro), has successfully landed its spacecraft on the Moon.
